In [1]:
from src.data.create_dataset import *

In [ ]:
df = read_data('2022', '02')

In [ ]:
df.info()

In [ ]:
## Preprocess (load data -> clean data -> save preprocessed data to cloud)
!python ../main/preprocess.py

In [ ]:
## Generate the features (generate features -> save the new dataframe with features to cloud)
!python ../main/feature_engineering.py

In [ ]:
## train the model (train the model -> save the model to cloud)
!python ../main/train_model.py